# Assignment 2 
### \<Zuxing Wu> \<a1816653>

## A. Tasks as specified for your team structure

**One person work: Information Retrieval and Extraction system**

### Read dataset

In [1]:
# %pip install pandas
import pandas as pd
# %pip install chardet
import chardet

# Detect the encoding of the file
rawdata = open('news_dataset.csv', 'rb').read()
result = chardet.detect(rawdata)
encoding = result['encoding']

data = pd.read_csv('news_dataset.csv', encoding=encoding)
data.head()

,id,author,date,year,month,topic,article
0,17307,Marlise Simons,1/01/2017,2017,1,architecture,PARIS ? When the Islamic State was about to...
1,17292,Andy Newman,31/12/2016,2016,12,art,Angels are everywhere in the Mu?iz family?s ap...
2,17298,Emma G. Fitzsimmons,2/01/2017,2017,1,business,Finally. The Second Avenue subway opened in Ne...
3,17311,Carl Hulse,3/01/2017,2017,1,business,WASHINGTON ? It?s or time for Republica...
4,17339,Jim Rutenberg,5/01/2017,2017,1,business,"For Megyn Kelly, the shift from Fox News to NB..."


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       1000 non-null   int64 
 1   author   994 non-null    object
 2   date     1000 non-null   object
 3   year     1000 non-null   object
 4   month    1000 non-null   object
 5   topic    1000 non-null   object
 6   article  1000 non-null   object
dtypes: int64(1), object(6)
memory usage: 54.8+ KB


In [3]:
data.count()

id         1000
author      994
date       1000
year       1000
month      1000
topic      1000
article    1000
dtype: int64

In [4]:
# keep only id, topic, article
data = data[['id', 'topic', 'article']]
data

,id,topic,article
0,17307,architecture,PARIS ? When the Islamic State was about to...
1,17292,art,Angels are everywhere in the Mu?iz family?s ap...
2,17298,business,Finally. The Second Avenue subway opened in Ne...
3,17311,business,WASHINGTON ? It?s or time for Republica...
4,17339,business,"For Megyn Kelly, the shift from Fox News to NB..."
...,...,...,...
995,18460,accidents,HONG KONG ? Hundreds of pilot whales that s...
996,18461,sports,"NICE, France ? Riv?re accepts the complim..."
997,18462,business,FRANKFURT ? Germans who never really warmed...
998,18463,sports,Charles Oakley has strong feelings about compe...


### Coreference Resolution function

In [5]:
# coreference resolution
# nerualcoref requires cython==0.29 and python 3.8
import spacy
%pip install cython==0.29
# !git clone https://github.com/huggingface/neuralcoref.git
# execute command after entering the directory
# %cd neuralcoref 
# !pip install -r requirements.txt 
# !pip install -e .
# !python -m spacy download en_core_web_sm

nlp = spacy.load('en_core_web_sm')

# load NeuralCoref and add it to the pipe of SpaCy's model
import neuralcoref
coref = neuralcoref.NeuralCoref(nlp.vocab)
nlp.add_pipe(coref, name='neuralcoref')

Note: you may need to restart the kernel to use updated packages.


In [6]:
# solve coreference in an article
# There should be no pronouns after this operation
def resolve_coref(article):
    return nlp(article)._.coref_resolved

In [7]:
# replace all '?' from article with empty string
data['article'] = data['article'].str.replace('?', '')
data

,id,topic,article
0,17307,architecture,PARIS When the Islamic State was about to ...
1,17292,art,Angels are everywhere in the Muiz familys apar...
2,17298,business,Finally. The Second Avenue subway opened in Ne...
3,17311,business,WASHINGTON Its or time for Republicans...
4,17339,business,"For Megyn Kelly, the shift from Fox News to NB..."
...,...,...,...
995,18460,accidents,HONG KONG Hundreds of pilot whales that sw...
996,18461,sports,"NICE, France Rivre accepts the complimen..."
997,18462,business,FRANKFURT Germans who never really warmed ...
998,18463,sports,Charles Oakley has strong feelings about compe...


In [8]:
# get test questions from test_questions_answers.txt
# Open the file in read mode ('r') using relative path
# %cd ../../
with open('test_questions_answers.txt', 'r') as file:
    # Read the content of the file
    test_questions = file.read()

print(test_questions)

('Who is the vice chairman of Samsung?', 17574) ('Jay Y. Lee', 24)
('Who was the President during the conflict?', 17311) ('George W. Bush', 24)
('Who is the Senator of Colorado?', 17311) ('Cory Gardner', 34)
('What was the revolt's name in 2010?', 17311) ('Tea Party Revolt', 25)
('When did Republicans get control back?', 17311) ('2010', 25)
('Where is the senior Republican from?', 17311) ('Oklahoma', 7)
('Who was the president during the Iraq War?', 17311) ('George W. Bush', 24)
('What amount did Fox News offer?', 17339) ('20 Million', 26)
('What show and program will Megyn Kelly host?', 17339) ('a daily daytime show and a Sunday newsmagazine program', 41)
('What is Andrew Lack adding?', 17339) ('journalist schooled in the preferences and worldviews of the conservative Americans who helped elect Mr Trump', 11)
('When is Donald Trump's inauguration?', 17339) ('Jan. 20', 2)
('Who is an executive chairman of 21st Century Fox?', 17339) ('Rupert Murdoch', 15)
('Is Douglas Brunt a novelist?'

### Handle questions

In [9]:
def get_question_answer(records):
    '''
    To extract question, answer, and article_id from the sample txt file
    '''
    records = records.split('\n')
    qa_pairs = []
    for record in records:
        r = record.split(') (')
        if len(r) == 2:  # Check if split operation produces two elements
            end = r[0].find("',")
            # Extract question and answer
            question = r[0][2 : end]
            article_id = r[0][end+3:]
            end = r[1].find("',")
            answer = r[1][1 : end]
            qa_pairs.append([question, answer, article_id])
    return qa_pairs

question_answer = get_question_answer(test_questions)
print(question_answer)

[['Who is the vice chairman of Samsung?', 'Jay Y. Lee', '17574'], ['Who was the President during the conflict?', 'George W. Bush', '17311'], ['Who is the Senator of Colorado?', 'Cory Gardner', '17311'], ["What was the revolt's name in 2010?", 'Tea Party Revolt', '17311'], ['When did Republicans get control back?', '2010', '17311'], ['Where is the senior Republican from?', 'Oklahoma', '17311'], ['Who was the president during the Iraq War?', 'George W. Bush', '17311'], ['What amount did Fox News offer?', '20 Million', '17339'], ['What show and program will Megyn Kelly host?', 'a daily daytime show and a Sunday newsmagazine program', '17339'], ['What is Andrew Lack adding?', 'journalist schooled in the preferences and worldviews of the conservative Americans who helped elect Mr Trump', '17339'], ["When is Donald Trump's inauguration?", 'Jan. 20', '17339'], ['Who is an executive chairman of 21st Century Fox?', 'Rupert Murdoch', '17339'], ['Is Douglas Brunt a novelist?', 'a novelist', '1733

### Prepare two models

In [10]:
# !pip install transformers
# %pip install torch
from transformers import BertForQuestionAnswering, BertTokenizer
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

# Load the QA model and tokenizer
model_name1 = "bert-large-cased-whole-word-masking-finetuned-squad"
model1 = BertForQuestionAnswering.from_pretrained(model_name1)
tokenizer1 = BertTokenizer.from_pretrained(model_name1)

model_name2 = "distilbert-base-cased-distilled-squad"
model2 = AutoModelForQuestionAnswering.from_pretrained(model_name2)
tokenizer2 = AutoTokenizer.from_pretrained(model_name2)

/Users/wzx/anaconda3/envs/nlp/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-large-cased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Define similarity check function

In [11]:
# Calculate the cosine similarity between query and article
# !pip install scikit-learn
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

def get_similarity(query, article):
    '''
    Calculate the cosine similarity between query and article
    '''
    # Combine the query and article into a list
    documents = [query, article]
    # Fit and transform the documents
    X = vectorizer.fit_transform(documents)
    # Calculate the cosine similarity
    cosine_sim = cosine_similarity(X[0:1], X[1:2])
    return cosine_sim[0][0]

### A function for picking the highest similarity sentence

In [12]:
# find the sentence with the highest similarity score
# !pip install nltk
import nltk

def get_best_sentence(query, article):
    # Tokenize the article into sentences
    sentences = nltk.sent_tokenize(article)
    best_sentence = ''
    best_score = 0
    for sentence in sentences:
        score = get_similarity(query, sentence)
        if score > best_score:
            best_score = score
            best_sentence = sentence

    print('Best sentence:', best_sentence, 'Similarity Score:', best_score)
    return best_sentence

### The 1st model to answer questions

In [13]:
def answer_question1(query, article):
    '''
    Takes a `query` string and an `article` string (which contains the
    answer), and identifies the words within the `article` that are the
    answer. Prints them out.
    '''
    # Resolve coreference in the article
    article = resolve_coref(article)
    # Check if the article is too long
    if len(article) > 512:
        # get the best sentence from the article
        best_sentence = get_best_sentence(query, article)
        # use the best sentence
        article = best_sentence

    # Tokenize the query and article
    input_ids = tokenizer1.encode(query, article)

    # Report how long the input sequence is.
    # print('Query has {:,} tokens.\n'.format(len(input_ids)))

    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer1.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # Truncate the sequence if it's longer than the limit
    max_length = 512
    if len(input_ids) > max_length:
        input_ids = input_ids[:max_length]
        segment_ids = segment_ids[:max_length]
        
    # ======== Evaluate ========
    # Run our example question through the model.
    model_scores = model1(torch.tensor([input_ids]), # The tokens representing our input text.
                                    token_type_ids=torch.tensor([segment_ids])) # The segment IDs to differentiate question from answer_text
    start_scores = model_scores.start_logits
    end_scores = model_scores.end_logits
    
    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)
    # get confidence score for this answer
    start_score = start_scores[0][answer_start].item()
    end_score = end_scores[0][answer_end].item()
    confidence_score = (start_score + end_score) / 2
    print('Answer Confidence Score:', confidence_score)
    # if confidence score is less than 0.5, return 'No answer'
    if confidence_score < 0.5:
        return 'No answer'

    # Get the string versions of the input tokens.
    tokens = tokenizer1.convert_ids_to_tokens(input_ids)

    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]
    #regulate the answer that is a name 
    answer = answer.replace(" .", ".")
    return answer

### The 2nd model to answer questions

In [14]:
def answer_question2(query, article):
    '''
    Takes a `query` string and an `article` string (which contains the
    answer), and identifies the words within the `article` that are the
    answer. Prints them out.
    '''
    # Resolve coreference in the article
    article = resolve_coref(article)
    # Check if the article is too long
    if len(article) > 512:
        # get the best sentence from the article
        best_sentence = get_best_sentence(query, article)
        # use the best sentence
        article = best_sentence
        
    # Encode the question and article using the tokenizer
    inputs = tokenizer2.encode_plus(query, article, return_tensors='pt', max_length=1024, truncation=True, truncation_strategy='only_second')
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    # Pass the encoded input through the QA model
    outputs = model2(input_ids, attention_mask=attention_mask, return_dict=True)
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

    # Decode the predicted start and end positions to get the answer
    start_index = torch.argmax(start_logits)
    end_index = torch.argmax(end_logits) + 1
    # get confidence score for this answer
    start_score = start_logits[0][start_index].item()
    end_score = end_logits[0][end_index].item()
    confidence_score = (start_score + end_score) / 2
    print('Answer Confidence Score:', confidence_score)
    # if confidence score is less than 0.5, return 'No answer'
    if confidence_score < 0.5:
        return 'No answer'

    # Skip over any tokens before the start position or after the end position
    for j, token_id in enumerate(input_ids[0]):
        if j < start_index or j >= end_index:
            input_ids[0][j] = tokenizer2.pad_token_id

    # Decode the answer from the corresponding tokens
    answer_tokens = input_ids[0][start_index:end_index]
    answer = tokenizer2.decode(answer_tokens)

    return answer

### Loop and answer all prepared questions

In [15]:
# search for the answer in the specific article
for qa in question_answer:
    question = qa[0]
    sample_answer = qa[1]
    article_id = qa[2]
    print("Question: ", question)
    print("Sample Answer: ", sample_answer)
    article = data[data['id'] == int(article_id)]['article'].values[0]
    similarity = get_similarity(question, article)
    print('Article Id:', article_id, ', Similarity Score:', similarity)

    print('Answer: "' + answer_question1(question, article) + '"')
    # print('Answer: "' + answer_question2(question, article) + '"')
    print("----------------------------------------------")

Question:  Who is the vice chairman of Samsung?
Sample Answer:  Jay Y. Lee
Article Id: 17574 , Similarity Score: 0.35718566250951606
Best sentence: Mr. Lee effectively runs Samsung, South Koreas largest conglomerate Mr. Lee is the son of SEOUL, South Korea     A special prosecutor investigating the corruption scandal that led to President Park  s impeachment chairman, Lee   who has been incapacitated with health problems. Similarity Score: 0.25847279483314756
Answer Confidence Score: 4.599022626876831
Answer: "Mr. Lee"
----------------------------------------------
Question:  Who was the President during the conflict?
Sample Answer:  George W. Bush
Article Id: 17311 , Similarity Score: 0.2874696945621706
Best sentence: This isnt the same style of Republican majority pushed from power after being routed in the 2006 midterm elections after the public backlash to the administration of President George W. Bush and his handling of the war in Iraq. Similarity Score: 0.2933823911756418
Answer

### Evaluation metrics (Exact match & F1 score)

In [16]:
# from transformers import pipeline
import numpy as np
from collections import Counter

# Load a pre-trained QA model and tokenizer
# qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")
# qa_pipeline = pipeline("question-answering", model="bert-large-cased-whole-word-masking-finetuned-squad")

# Function to compute exact match
def exact_match(prediction, expected_answer):
    return int(any(prediction == w for w in expected_answer))

# Function to compute F1 score
def f1_score(prediction, expected_answer):
    def compute_f1(a_gold, a_pred):
        gold_tokens = a_gold.split()
        pred_tokens = a_pred.split()
        common = Counter(gold_tokens) & Counter(pred_tokens)
        num_same = sum(common.values())
        if num_same == 0:
            return 0
        precision = 1.0 * num_same / len(pred_tokens)
        recall = 1.0 * num_same / len(gold_tokens)
        f1 = (2 * precision * recall) / (precision + recall)
        return f1

    return max(compute_f1(w, prediction) for w in expected_answer)

### Show general scores by calculating the mean values

In [19]:
em_scores = []
f1_scores = []

# Evaluate the QA model on the test questions
for qa in question_answer:
    question = qa[0]
    expected_answer = [qa[1]]
    article_id = qa[2]
    article = data[data['id'] == int(article_id)]['article'].values[0]
    result = answer_question1(query=question, article=article)
    # result = answer_question2(query=question, article=article)
    em_score = exact_match(result, expected_answer)
    f1 = f1_score(result, expected_answer)
    em_scores.append(em_score)
    f1_scores.append(f1)

# Compute the average EM and F1 scores
average_em = np.mean(em_scores)
average_f1 = np.mean(f1_scores)

print(f"Average Exact Match: {average_em}")
print(f"Average F1 Score: {average_f1}")

# when using "distilbert-base-cased-distilled-squad" model, results are:
# Average Exact Match: 0.28
# Average F1 Score: 0.47248085248085253
# when using "bert-large-cased-whole-word-masking-finetuned-squad" model, results are:
# Average Exact Match: 0.34
# Average F1 Score: 0.5421409897292251

Best sentence: Mr. Lee effectively runs Samsung, South Koreas largest conglomerate Mr. Lee is the son of SEOUL, South Korea     A special prosecutor investigating the corruption scandal that led to President Park  s impeachment chairman, Lee   who has been incapacitated with health problems. Similarity Score: 0.25847279483314756
Answer Confidence Score: 4.599022626876831
Best sentence: This isnt the same style of Republican majority pushed from power after being routed in the 2006 midterm elections after the public backlash to the administration of President George W. Bush and his handling of the war in Iraq. Similarity Score: 0.2933823911756418
Answer Confidence Score: 9.57399606704712
Best sentence: Republicans who have shied from the responsibility of government will now be called upon to support increases in the debt limit, approve annual budgets, endorse spending bills and back other   measures that Republicans who have shied from the responsibility of government formerly left to 

## B. References

1. Most code are modified on the basis of workshop code.
2. Some doubts are solved by asking ChatGPT.
3. Useful webpage:  
https://huggingface.co/learn/nlp-course/chapter7/7?fw=pt
4. Copilot is used when coding and debugging
5. Useful links:  
https://github.com/huggingface/neuralcoref/issues/357,  
https://stackoverflow.com/questions/61269954/attribute-error-using-neuralcoref-in-colab

## C. Appendix

Since I can't submit a .txt file, I just put questions and answers here:

('Who is the vice chairman of Samsung?', 17574) ('Jay Y. Lee', 24)
('Who was the President during the conflict?', 17311) ('George W. Bush', 24)
('Who is the Senator of Colorado?', 17311) ('Cory Gardner', 34)
('What was the revolt's name in 2010?', 17311) ('Tea Party Revolt', 25)
('When did Republicans get control back?', 17311) ('2010', 25)
('Where is the senior Republican from?', 17311) ('Oklahoma', 7)
('Who was the president during the Iraq War?', 17311) ('George W. Bush', 24)
('What amount did Fox News offer?', 17339) ('20 Million', 26)
('What show and program will Megyn Kelly host?', 17339) ('a daily daytime show and a Sunday newsmagazine program', 41)
('What is Andrew Lack adding?', 17339) ('journalist schooled in the preferences and worldviews of the conservative Americans who helped elect Mr Trump', 11)
('When is Donald Trump's inauguration?', 17339) ('Jan. 20', 2)
('Who is an executive chairman of 21st Century Fox?', 17339) ('Rupert Murdoch', 15)
('Is Douglas Brunt a novelist?', 17339) ('a novelist', 24)
('Which city and Which Province was the shooting?', 17344) ('Panzhihua city, Sichuan Province', 2)
('Who was embarrassed by the violence?', 17344) ('Xi Jinping', 4)
('Chen Zhongshu is the head of what?', 17344) ('Panzhihua Land and Resources Bureau,', 6)
('The shooting happened where?', 17344) ('a exhibition center', 2)
('Which year did Zhang Yan start working there?', 17344) ('2006', 11)
('Where is Panzhihua, which Province?', 17344) ('Sichuan Province', 2)
('What kind of city is Panzhihua?', 17344) ('industrial city', 2)
('What news app did Apple remove from its app store by the request of Chinese authorities?', 17368) ('The New York Times', 6)
('What kind of job did Hans Rosling do?', 18465) ('doctor', 0)
('Where did Hans Rosling die?', 18465) ('Uppsala, Sweden', 0)
('When did Hans Rosling die?', 18465) ('Tuesday', 0)
('How old was Hans Rosling when he died?', 18465) ('68', 0)
('Is Hans Rosling still alive?', 18465) ('died on Tuesday in Uppsala, Sweden. He was 68', 0)
('Where did Yves Ubelmann get a call from?', 17307) ('Syria', 0)
('How old is Mr. Ubelmann?', 17307) ('36', 0)
('Who did they get the permisson from for getting married?', 17292) ('her uncle, a judge', 0)
('In which city did The Second Avenue subway open?', 17298) ('New York City', 0)
('When was The Second Avenue first proposed?', 17298) ('1920s', 0)
('When did Betsy Morris leave the 96th Street sation?', 17298) ('at noon', 0)
('When was his work included in group shows?', 17285) ('1932 and 1934', 0)
('How many shows did they do for Thierry Mugler in Paris?', 17308) ('five', 0)
('When is Mr. Mulvaney's confirmation hearing is scheduled?', 17760) ('Jan. 24', 0)
('What are the Knicks celebrating?', 18463) ('their 70th anniversary', 0)
('Has Oakley been invited to any of the events?', 18463) ('No', 0)
('How many seasons has Oakley played for the Knicks?', 18463) ('10', 0)
('Did Dolan become the chairman of the Graden before Oakley's left?', 18463) ('No', 0)
('What did a teammate urged Oakley to do?', 18463) ('to keep his voice down', 0)
('Why did West Germany move gold abroad?', 18462) ('out of fear they would be seized by an invading Soviet army', 0)
('How many gold bars had been transfered by plan according to Bundesbank's words?', 18462) ('$13 billion', 0)
('When did the Belin Wall fall?', 18462) ('1989', 0)
('What's the form of the only sold gold by Bundesbank??', 18462) ('commemorative coins', 0)
('How many nuclear tests has North Korea conducted in the last decade?', 17287) ('five', 0)
('How many ballistic missile tests in 2016 alone?', 17287) ('20', 0)
('How far could a warhead of 1,100 to 1,300 pounds fly?', 17287) ('more than 7,400 miles', 0)
('What test has North Korea complished in April?', 17287) ('successful ground test of an engine for an intercontinental ballistic missile', 0)
('When did the UN Security Council impose new sanctiions against North Korea?', 17287) ('In November', 0)
('Who is the leader of North Korea?', 17287) ('Kim', 0)
